In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import KNNImputer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import optuna

In [ ]:
# 데이터 로드
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# ID 제거
df_train.drop(columns=['ID'], inplace=True)
df_test_ids = df_test['ID']
df_test.drop(columns=['ID'], inplace=True)

In [ ]:
# 타겟 변수 분리
X = df_train.drop(columns=['임신 성공 여부'])
y = df_train['임신 성공 여부']

In [ ]:
from sklearn.impute import SimpleImputer

# 숫자형 변수 결측치 평균으로 대체
simple_imputer = SimpleImputer(strategy="mean")  # 또는 "median" (중앙값 사용 가능)

X[numeric_features] = simple_imputer.fit_transform(X[numeric_features])
df_test[numeric_features] = simple_imputer.transform(df_test[numeric_features])

In [ ]:
# 2. 범주형 변수 인코딩
categorical_features = X.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[categorical_features] = encoder.fit_transform(X[categorical_features])
df_test[categorical_features] = encoder.transform(df_test[categorical_features])

In [ ]:
# 3. Feature Scaling (StandardScaler 적용)
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])
df_test[numeric_features] = scaler.transform(df_test[numeric_features])

In [ ]:
# 4. Feature Selection (LightGBM 기반)
lgbm = LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1)
lgbm.fit(X, y)
feature_importance = pd.Series(lgbm.feature_importances_, index=X.columns).sort_values(ascending=False)
selected_features = feature_importance[feature_importance > 0.001].index.tolist()
X = X[selected_features]
df_test = df_test[selected_features]

In [ ]:
# 훈련 / 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 5. Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_val_pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, y_val_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
best_params = study.best_params
print(f"🔹 Best Params: {best_params}")

In [ ]:
# 6. 최적 하이퍼파라미터를 적용하여 모델 학습
lgbm_best = LGBMClassifier(**best_params)
lgbm_best.fit(X_train, y_train)


# 🔹 검증 데이터 평가 (Validation ROC-AUC)
y_val_pred = lgbm_best.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f'✅ Validation ROC-AUC: {roc_auc:.4f}')

In [ ]:
# 🔹 테스트 데이터 예측
test_preds = lgbm_best.predict_proba(df_test)[:, 1]

In [ ]:
# 7. 제출 파일 생성
submission = pd.DataFrame({'ID': df_test_ids, 'probability': test_preds})
submission.to_csv('submission_5.csv', index=False)
print("✅ Submission file saved: submission_5.csv")